<a href="https://colab.research.google.com/github/sisam1234/Book-Recommendation-System-Using-KNN/blob/main/book_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'


--2024-09-20 14:22:45--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M   103MB/s    in 0.2s    

2024-09-20 14:22:45 (103 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

df_ori = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author','year','publisher','img'],
    usecols=['isbn', 'title', 'author','year','publisher','img'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str','year':'str','publisher':'str','img':'str'})




In [ ]:
user_rating=df_ratings['user'].value_counts()>200
user_rating=user_rating[user_rating].index
user_rating

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int32', name='user', length=899)

In [ ]:
filtered_rating=df_ratings[df_ratings['user'].isin(user_rating)]
filtered_rating

,user,isbn,rating
1456,277427,002542730X,10.0
1457,277427,0026217457,0.0
1458,277427,003008685X,8.0
1459,277427,0030615321,0.0
1460,277427,0060002050,0.0
...,...,...,...
1147612,275970,3829021860,0.0
1147613,275970,4770019572,0.0
1147614,275970,896086097,0.0
1147615,275970,9626340762,8.0


In [ ]:
book_rating=df_ratings['isbn'].value_counts()>100
b_rating=book_rating[book_rating].index
b_rating

Index(['0971880107', '0316666343', '0385504209', '0060928336', '0312195516',
       '044023722X', '0679781587', '0142001740', '067976402X', '0671027360',
       ...
       '080411918X', '0140042598', '0446604844', '076790592X', '0385508417',
       '0446530522', '0385335830', '0312924801', '051511992X', '0553208845'],
      dtype='object', name='isbn', length=721)

In [ ]:
filtered_rating=filtered_rating[filtered_rating['isbn'].isin(b_rating)]
filtered_rating

,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0
...,...,...,...
1147304,275970,0804111359,0.0
1147436,275970,140003065X,0.0
1147439,275970,1400031346,0.0
1147440,275970,1400031354,0.0


In [ ]:
filtered_rating=filtered_rating.drop_duplicates(["isbn","user"])
filtered_rating

,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0
...,...,...,...
1147304,275970,0804111359,0.0
1147436,275970,140003065X,0.0
1147439,275970,1400031346,0.0
1147440,275970,1400031354,0.0


In [ ]:
df=filtered_rating.merge(df_ori,on="isbn")
df

In [ ]:
pt=df.pivot_table(values='rating',index='title',columns='user')
pt

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pt.fillna(0,inplace=True)
pt

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Without Remorse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_matrix=csr_matrix(pt.values)
df_matrix

<664x882 sparse matrix of type '<class 'numpy.float32'>'
	with 12327 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
nn=NearestNeighbors(algorithm='brute',metric='cosine',p=2)
nn.fit(df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
book_name = pt.index
book_name

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       'A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash',
       'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Confederacy of Dunces (Evergreen Book)',
       ...
       'Wicked: The Life and Times of the Wicked Witch of the West', 'Wifey',
       'Wild Animus', 'Winter Moon', 'Wish You Well', 'Without Remorse',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=664)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pickle
pickle.dump(nn,open('model/model.pkl','wb'))
pickle.dump(book_name,open('model/book_name.pkl','wb'))
pickle.dump(df,open('model/datasets.pkl','wb'))
pickle.dump(pt,open('model/book_pivot.pkl','wb'))

FileNotFoundError: [Errno 2] No such file or directory: 'model/model.pkl'

In [ ]:
def get_recommends(book = ""):
  recommended_books=[book]
  dis,book_info=nn.kneighbors([pt.loc[book]],n_neighbors=6, return_distance=True)
  recom=pt.iloc[np.flip(book_info[0])[:-1]].index.to_list()
  recomd=list(np.flip(dis[0])[:-1])
  for r in  zip(recom,recomd):
    recommended_books[1].append(list(r))
  return recommended_books

In [ ]:
def get_recommends(book=""):
    recommended_books = [book]
    dis, book_info = nn.kneighbors([pt.loc[book]], n_neighbors=6, return_distance=True)
    recom = pt.iloc[np.flip(book_info[0])[:-1]].index.to_list()
    recomd = list(np.flip(dis[0])[:-1])

    # Adding recommended books and their distances to the list
    for r in zip(recom, recomd):
        recommended_books.append(list(r))

    return recommended_books



In [ ]:
def get_recommends(book=""):
    recommended_books = [book]
    dis, book_info = nn.kneighbors([pt.loc[book]], n_neighbors=6, return_distance=True)
    recom = pt.iloc[np.flip(book_info[0])[:-1]].index.to_list()

    # Adding only the recommended book names to the list
    recommended_books.extend(recom)

    return recommended_books


In [ ]:
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")



TypeError: 'int' object is not iterable

In [ ]:
book = get_recommends("Manhattan Hunt Club")
print(book)

['Manhattan Hunt Club', [['Dust to Dust', 0.7525964], ['Debt of Honor (Jack Ryan Novels)', 0.74025726], ['The Blue Nowhere : A Novel', 0.7401512], ['Shopgirl', 0.70460415], ['The Bear and the Dragon (Jack Ryan Novels)', 0.63190836]]]
